In [1]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
from ortools.constraint_solver import pywrapcp

def find_cheap_flights(prices, durations):
    num_flights = len(prices)

    # Create the solver
    solver = pywrapcp.Solver("cheap_flights")

    # Create variables
    selected_flights = [solver.BoolVar(f'flight_{i}') for i in range(num_flights)]

    # Create constraints
    solver.Add(solver.Sum(selected_flights) > 0)  # At least one flight should be selected

    total_price = solver.Sum(selected_flights[i] * prices[i] for i in range(num_flights))
    total_duration = solver.Sum(selected_flights[i] * durations[i] for i in range(num_flights))
    solver.Add(total_duration <= 10)  # Total duration should be less than or equal to 10

    # Create objective
    objective = solver.Minimize(total_price, 1)

    db = solver.Phase(selected_flights,
                      solver.CHOOSE_FIRST_UNBOUND,
                      solver.ASSIGN_MIN_VALUE)

    # Create solution collector
    solution = solver.Assignment()
    solution.Add(selected_flights)
    collector = solver.AllSolutionCollector(solution)

    # Solve the problem
    solver.Solve(db, [objective, collector])

    if collector.SolutionCount() > 0:
        print("Selected Flights:")
        for i in range(collector.SolutionCount()):
            selected_flight_indices = [j for j in range(num_flights) if collector.Value(i, selected_flights[j]) == 1]
            print(f"Solution {i + 1}: {selected_flight_indices}")
            print("Total Price:", collector.Value(i, total_price))
            print("Total Duration:", collector.Value(i, total_duration))
            print()

    else:
        print("No solution found.")


# Example usage
prices = [100, 200, 150, 300, 250]
durations = [2, 3, 2, 4, 3]

find_cheap_flights(prices, durations)

In [3]:
from ortools.constraint_solver import pywrapcp

def find_cheap_flights(prices, durations):
    num_flights = len(prices)

In [6]:
# Create the solver
    solver = pywrapcp.Solver("cheap_flights")

    # Create variables
    selected_flights = [solver.BoolVar(f'flight_{i}') for i in range(num_flights)]

    # Create constraints
    solver.Add(solver.Sum(selected_flights) > 0)  # At least one flight should be selected

    total_price = solver.Sum(selected_flights[i] * prices[i] for i in range(num_flights))
    total_duration = solver.Sum(selected_flights[i] * durations[i] for i in range(num_flights))
    solver.Add(total_duration <= 10)  # Total duration should be less than or equal to 10

    # Create objective
    objective = solver.Minimize(total_price, 1)

    db = solver.Phase(selected_flights,
                      solver.CHOOSE_FIRST_UNBOUND,
                      solver.ASSIGN_MIN_VALUE)

    # Create solution collector
    solution = solver.Assignment()
    solution.Add(selected_flights)
    collector = solver.AllSolutionCollector(solution)

    # Solve the problem
    solver.Solve(db, [objective, collector])

IndentationError: ignored

In [1]:
from ortools.linear_solver import pywraplp

def find_cheap_tickets():
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Variables
    flights = ['Flight1', 'Flight2', 'Flight3']  # Example flights
    prices = {
        'Flight1': [100, 200, 300],
        'Flight2': [150, 250, 350],
        'Flight3': [200, 300, 400]
    }


    ticket_vars = {}
    for flight in flights:
        ticket_vars[flight] = solver.BoolVar(f'{flight}_ticket')

    # Set the objective function
    objective = solver.Objective()
    for flight in flights:
        for i, price in enumerate(prices[flight]):
            objective.SetCoefficient(ticket_vars[flight], price)

    objective.SetMinimization()

    for flight in flights:
        constraint = solver.Constraint(0, 1)  # Select at most one ticket per flight
        constraint.SetCoefficient(ticket_vars[flight], 1)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        cheapest_price = sum(ticket_vars[flight].solution_value() * min(prices[flight]) for flight in flights)
        cheapest_ticket = {flight: min(prices[flight]) for flight in flights if ticket_vars[flight].solution_value() == 1}
        return cheapest_price, cheapest_ticket
    else:
        return None, None

cheapest_price, cheapest_ticket = find_cheap_tickets()

if cheapest_price is not None:
    print("Cheapest price:", cheapest_price)
    print("Cheapest ticket:", cheapest_ticket)
else:
    print("No feasible solution found.")

Cheapest price: 0.0
Cheapest ticket: {}


In [2]:
from ortools.sat.python import cp_model

def find_cheap_flights(prices, durations):
    num_flights = len(prices)

    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Create variables
    selected_flights = [model.NewBoolVar(f'flight_{i}') for i in range(num_flights)]

    # Add constraint: at least one flight selected
    model.Add(sum(selected_flights) >= 1)

    # Add constraint: total duration should be less than or equal to 10
    total_duration = sum(selected_flights[i] * durations[i] for i in range(num_flights))
    model.Add(total_duration <= 10)

    objective = sum(selected_flights[i] * prices[i] for i in range(num_flights))
    model.Minimize(objective)

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        selected_flight_indices = [i for i in range(num_flights) if solver.Value(selected_flights[i]) == 1]
        total_price = sum(prices[i] for i in selected_flight_indices)
        cheapest_ticket = {f'flight_{i}': solver.Value(selected_flights[i]) for i in range(num_flights)}

        print("Selected flights:", selected_flight_indices)
        print("Cheapest price:", total_price)
        print("Cheapest ticket:", cheapest_ticket)
    else:
        print("No solution found.")


# Example usage
prices = [100, 200, 150, 300, 250]
durations = [2, 3, 2, 4, 3]

find_cheap_flights(prices, durations)








Selected flights: [0]
Cheapest price: 100
Cheapest ticket: {'flight_0': 1, 'flight_1': 0, 'flight_2': 0, 'flight_3': 0, 'flight_4': 0}
